In [ ]:
# 예측을 수행하기 위해 모든 user-item에 대한 예측값을 계산
p = np.matmul(O.numpy(), np.transpose(U.numpy())) + b.numpy()
# user_category_not_valid에 해당하지 않는 경우에 대해 precision, recall, f1_score 계산
# 평균을 위한 초기화
precision_m, recall_m, f1_score_m, count_m = 0, 0, 0, 0
for i in range(n_u):
    for category in df_limit['평균기온(°C)'].unique():
        
        # 실제 온도
        # 평균을 적용하고 temp를 빼서 값이 작을수록 실제 온도에 가깝도록 함. 이 때 각 user-item의 사용 횟수를 가중하여 많이 사용한 item이 추천되도록 함
        pm = np.power(p + o_mean - category, 2)  -count * 28
        my_predictions = pm[:,i]

        # sort predictions
        ix = tf.argsort(my_predictions, direction='ASCENDING')

        df_predict = UI_temp[UI_temp.columns[ix[0:3]]].copy()
        df_predict = df_predict.round(0)
        # df_predict의 columns와 test_data_df의 '옷 조합' column을 비교하여 일치하는 경우의 개수를 계산
        predict = df_predict.columns.astype(str)
        
        # user i에 대한 예측을 파일로 저장
        os.makedirs(f'../data/predictions/male/user_{i+1}', exist_ok=True)
        # Save predictions to file in user's directory
        with open(f'../data/predictions/male/user_{i+1}/predictions_{category}.txt', 'w') as f:
            for item in predict:
                f.write("%s\n" % item)
        
        if i+1 in user_category_not_valid and category in user_category_not_valid[i+1]:
            print(f'{i+1}번 user, {category}도 데이터가 부족하여 제외합니다.')
            continue
        
        count_m += 1
        
        label = test_data_df[(test_data_df['userId'] == i+1) & (test_data_df['평균기온(°C)'] == category)]['옷 조합'].astype(str)
        
        precision = len(set(predict) & set(label)) / len(set(predict))
        print(f'{i+1}번 user, {category}도 prediction: {predict}')
        print(f'{i+1}번 user, {category}도 label: {label}')
        recall = len(set(predict) & set(label)) / len(set(label))
        if precision + recall == 0:
            '''print(f'0인 경우')
            print(f'{i+1}번 user, {category}도 예측 결과: {predict}, 실제 결과: {label} ')'''  
            f1_score = 0
        else:
            '''print(f'0이 아닌 경우')
            print(f'{i+1}번 user, {category}도 예측 결과: {predict}, 실제 결과: {label} ')'''  
            f1_score = 2 * (precision * recall) / (precision + recall)
        precision_m += precision
        recall_m += recall
        f1_score_m += f1_score
        print(precision, recall, f1_score)
precision_m /= count_m
recall_m /= count_m
f1_score_m /= count_m
print(f'평균 precision: {precision_m}, 평균 recall: {recall_m}, 평균 f1_score: {f1_score_m}')